In [34]:
import openslide
import numpy as np
import scipy.misc
import scipy.io
import cv2
import os
import matplotlib.pyplot as plt    
from skimage import measure
import logging
from multiprocessing import Pool

In [35]:
# 获取切片单元的上下y值
def RemoveHeadEnd(OriImg):
    n, m, z = OriImg.shape
    i = 0
    while np.sum(OriImg[i, :, :]) == 0 and i < n-1:
        i += 1
    h = i
    j = n - 1
    while np.sum(OriImg[j, :, :]) == 0 and j > 0:
        j -= 1
    e = j
    return h, e

In [36]:
# 对图片做处理：使得信息量高的值为1，低的为0
def get_result(img):
    bw = np.where(np.logical_and(img[:, :, 0] > 130, np.logical_or(img[:, :, 0] - img[:, :, 1] > 30,
                                                                   img[:, :, 0] - img[:, :, 2] > 30)), 1, 0)
    bw = bw.astype(dtype=np.float32)
    img_Blur = cv2.blur(bw, (100, 100), cv2.BORDER_REPLICATE)
    result = img_Blur
    result[result > 0.4] = 1
    result[result <= 0.4] = 0
    return result

In [37]:
# 获得每一个连通域，并保存各连通域内的肾小球
def save_glomerular(slide, each, label_image, unit_path):
    for region in measure.regionprops(label_image):
        pos = region.bbox
        if pos[2]-pos[0]<10 or pos[3]-pos[1]<10:
            continue
        else:
            site = (each[0][0]+pos[1]*zoom_ratio, each[0][1]+pos[0]*zoom_ratio)
            dimension = (pos[3]*zoom_ratio-pos[1]*zoom_ratio, pos[2]*zoom_ratio-pos[0]*zoom_ratio)
            result = np.array(slide.read_region(site,0,dimension))
            scipy.misc.imsave(unit_path+'/{}.tif'.format(measure.regionprops(label_image).index(region)+1), result)

In [38]:
if not os.path.exists('./SegPic'):
    os.mkdir('./SegPic')
svsFiles_dir = '/media/frodo/Seagate Expansion Drive/temp/all material/SVSFiles/'
MaskPath = '/home/frodo/Desktop/job content/所有mask/'
svsFiles_names = os.listdir(svsFiles_dir)
svsFiles_names.sort()

In [39]:
maskname_list = []
for maskname in os.listdir(MaskPath):
    name = maskname.split('_')[0]
    maskname_list.append(name)
maskname_set = set(maskname_list)

In [41]:
for imgname in svsFiles_names:
#     imgname = '9273.svs' 
    
    # 如果mask文件夹里面没有对应svs的mask图片，则跳过本次循环
    if imgname not in maskname_set:
        continue

    # 如果有mask图片，则继续判断判断是否已经存在对应的文件夹，没有则创建
    if not os.path.exists('./SegPic/' + imgname):
        os.mkdir('./SegPic/' + imgname)

    # 定义一个存放元祖（坐标，宽高）的列表    
    loc_dim = []
   
    # 利用openslide读取图片
    slide = openslide.open_slide(svsFiles_dir + imgname)
    # 获得对应图像大小下的缩放倍率
    zoom_ratio = int(slide.level_downsamples[1])
    
    # 读取区背景后的图片
    BackRemImg = scipy.io.loadmat('/media/frodo/Seagate Expansion Drive/temp/all material/SegPic/{}/去背景图.mat'.format(imgname))['BackRem']
    n, m, z = BackRemImg.shape
    result = np.where(BackRemImg, 1, 0)[:,:,0]

    # 生成（1,m）的数组，为图片的宽（x轴）
    PatchSegMark = np.zeros((1, m))
    # 如果图片的每一单位像素的行的值的总和不为0,说明有切片单元，值设为1；反之，说明没有切片单元，则该位置值设为0
    for i in range(m):
        if np.sum(result[:, i]):
            PatchSegMark[0, i] = 1
        else:
            PatchSegMark[0, i] = 0
    # 对相邻的值做差离值，若值为1（1-0）或者-1（0-1），说明这是切片单元的边界处
    PatchSegPos = np.diff(PatchSegMark, n=1)
    # 值为1代表切片单元的起始边界
    PatchSegPosStart = np.where(PatchSegPos == 1)[1]
    # 值为-1代表切片单元的结束边界
    PatchSegPosEnd = np.where(PatchSegPos == -1)[1]
    # 获得起始边界和结束边界的个数
    StartNum = len(PatchSegPosStart)
    EndNum = len(PatchSegPosEnd)

    # 确定切片单元的位置
    if PatchSegPosStart[0] > PatchSegPosEnd[0]:         # 代表先出现切片单元
        patch = BackRemImg[:, :PatchSegPosEnd[0]+1, :]    # 第一张图片在0～第一个切片单元的结束边界选取
        h, e = RemoveHeadEnd(patch)                     # 获得每个切片单元的上下边界
        loc_dim.append(((0*zoom_ratio,h*zoom_ratio),((PatchSegPosEnd[0])*zoom_ratio, (e-h)*zoom_ratio)))

        if StartNum < EndNum:
            for i in range(StartNum):
                patch = BackRemImg[:, PatchSegPosStart[i]:PatchSegPosEnd[i + 1]+1, :]
                h, e = RemoveHeadEnd(patch)
                if e > h:
                    loc_dim.append(((PatchSegPosStart[i] *zoom_ratio, h*zoom_ratio), ((PatchSegPosEnd[i + 1] - PatchSegPosStart[i])*zoom_ratio, (e-h)*zoom_ratio)))
        else:
            for i in range(StartNum - 1):
                patch = BackRemImg[:, PatchSegPosStart[i]:PatchSegPosEnd[i + 1]+1, :]
                h, e = RemoveHeadEnd(patch)
                loc_dim.append(((PatchSegPosStart[i]*zoom_ratio,h*zoom_ratio),((PatchSegPosEnd[i + 1] -PatchSegPosStart[i])*zoom_ratio, (e-h)*zoom_ratio)))

            patch = BackRemImg[:, PatchSegPosStart[StartNum - 1]:m+1, :]
            h, e = RemoveHeadEnd(patch)
            loc_dim.append(((PatchSegPosStart[StartNum - 1]*zoom_ratio,h*zoom_ratio),((m-PatchSegPosStart[StartNum - 1])*zoom_ratio,(e-h)*zoom_ratio)))
    else:
        if StartNum > EndNum:
            for i in range(StartNum - 1):
                patch = BackRemImg[:, PatchSegPosStart[i]:PatchSegPosEnd[i]+1, :]
                h, e = RemoveHeadEnd(patch)
                loc_dim.append(((PatchSegPosStart[i]*zoom_ratio,h*zoom_ratio),((PatchSegPosEnd[i] -PatchSegPosStart[i])*zoom_ratio, (e-h)*zoom_ratio)))
            patch = BackRemImg[:, PatchSegPosStart[StartNum - 1]:m+1, :]
            h, e = RemoveHeadEnd(patch)
            loc_dim.append(((PatchSegPosStart[StartNum - 1]*zoom_ratio,h*zoom_ratio),((m-PatchSegPosStart[StartNum - 1])*zoom_ratio,(e-h)*zoom_ratio)))
        else:
            for i in range(StartNum):
                patch = BackRemImg[:, PatchSegPosStart[i]:PatchSegPosEnd[i]+1, :]
                h, e = RemoveHeadEnd(patch)
#                 if e > h:
                loc_dim.append(((PatchSegPosStart[i]*zoom_ratio,h*zoom_ratio),((PatchSegPosEnd[i] -PatchSegPosStart[i])*zoom_ratio,(e-h)*zoom_ratio)))

    # 根据低分辨率的切片单元的位置，判定出高分辨率下对应切片单元的位置，进行切片
    print(imgname, ' started!')
#     print(len(loc_dim))
    for each in loc_dim:
        i = loc_dim.index(each) + 1
        # 读取mask图片
        img_mask_path = MaskPath + imgname + '_' + str(i)+'.mask.png.bmp'
        bmp = cv2.imread(img_mask_path, 0)
        if bmp is not None:
            # 为当前图片的第n个切片单元创建对应的文件夹
            unit_path = 'SegPic/'+imgname+'/'+str(i)      # 当前切片单元路径
            if not os.path.exists(unit_path):
                os.mkdir(unit_path)

            bw = np.where(bmp, 1, 0)        # 生成阈值
            label_image =measure.label(bw)  # 连通区域标记
            # 获得每一个连通域，并且保存肾小球
            save_glomerular(slide, each, label_image, unit_path)
#         break
    print(imgname, ' finished! \n')
#     break

8602.svs  started!


/home/frodo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  # This is added back by InteractiveShellApp.init_path()


8602.svs  finished! 

8604.svs  started!
8604.svs  finished! 

8607.svs  started!
8607.svs  finished! 

8608.svs  started!
8608.svs  finished! 

8610.svs  started!
8610.svs  finished! 

8611.svs  started!
8611.svs  finished! 

8612.svs  started!
8612.svs  finished! 

8613.svs  started!
8613.svs  finished! 

8616.svs  started!
8616.svs  finished! 

8619.svs  started!
8619.svs  finished! 

8622.svs  started!
8622.svs  finished! 

8623.svs  started!
8623.svs  finished! 

8626.svs  started!
8626.svs  finished! 

8628.svs  started!
8628.svs  finished! 

8629.svs  started!
8629.svs  finished! 

8630.svs  started!
8630.svs  finished! 

8631.svs  started!
8631.svs  finished! 

8632.svs  started!
8632.svs  finished! 

8633.svs  started!
8633.svs  finished! 

8635.svs  started!
8635.svs  finished! 

8636.svs  started!
8636.svs  finished! 

8637.svs  started!
8637.svs  finished! 

8638.svs  started!
8638.svs  finished! 

8640.svs  started!
8640.svs  finished! 

8643.svs  started!
8643.svs  finish

8906.svs  finished! 

8907.svs  started!
8907.svs  finished! 

8908.svs  started!
8908.svs  finished! 

8909.svs  started!
8909.svs  finished! 

8912.svs  started!
8912.svs  finished! 

8914.svs  started!
8914.svs  finished! 

8916.svs  started!
8916.svs  finished! 

8917.svs  started!
8917.svs  finished! 

8918.svs  started!
8918.svs  finished! 

8920.svs  started!
8920.svs  finished! 

8921.svs  started!
8921.svs  finished! 

8922.svs  started!
8922.svs  finished! 

8923.svs  started!
8923.svs  finished! 

8924.svs  started!
8924.svs  finished! 

8925.svs  started!
8925.svs  finished! 

8926.svs  started!
8926.svs  finished! 

8930.svs  started!
8930.svs  finished! 

8933.svs  started!
8933.svs  finished! 

8934.svs  started!
8934.svs  finished! 

8935.svs  started!
8935.svs  finished! 

8937.svs  started!
8937.svs  finished! 

8938.svs  started!
8938.svs  finished! 

8939.svs  started!
8939.svs  finished! 

8942.svs  started!
8942.svs  finished! 

8944.svs  started!
8944.svs  finish

9208.svs  finished! 

9209.svs  started!
9209.svs  finished! 

9211.svs  started!
9211.svs  finished! 

9213.svs  started!
9213.svs  finished! 

9214.svs  started!
9214.svs  finished! 

9215.svs  started!
9215.svs  finished! 

9219.svs  started!
9219.svs  finished! 

9220.svs  started!
9220.svs  finished! 

9221.svs  started!
9221.svs  finished! 

9222.svs  started!
9222.svs  finished! 

9223.svs  started!
9223.svs  finished! 

9262.svs  started!
9262.svs  finished! 

9264.svs  started!
9264.svs  finished! 

9265.svs  started!
9265.svs  finished! 

9267.svs  started!
9267.svs  finished! 

9268.svs  started!
9268.svs  finished! 

9269.svs  started!
9269.svs  finished! 

9270.svs  started!
9270.svs  finished! 

9271.svs  started!
9271.svs  finished! 

9272.svs  started!
9272.svs  finished! 

9273.svs  started!
9273.svs  finished! 

9275.svs  started!
9275.svs  finished! 

9278.svs  started!
9278.svs  finished! 

9280.svs  started!
9280.svs  finished! 

9282.svs  started!
9282.svs  finish

# 下面的是试验：将openslide读取出来的图片直接转换成RGB格式

In [13]:
a = openslide.open_slide('/media/frodo/Seagate Expansion Drive/temp/all material/SVSFiles/8734.svs')
a.level_dimensions

((73704, 28366), (18426, 7091), (4606, 1772), (2303, 886))

In [15]:
res1 = np.array(slide.read_region((0,0),1,(18426, 7091)))
res1 = np.array(slide.read_region((0,0),1,(18426, 7092)))

In [13]:
# 可以在读取的过程中转换成RGB格式
res2 = np.array(slide.read_region((0,0),1,a.level_dimensions[1]).convert('RGB'))
np.sum(res2过程中

80092220383